In [5]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
from torch.optim.lr_scheduler import StepLR

In [6]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 32, 3, 1)
        self.conv2 = nn.Conv2d(32, 64, 3, 1)
        self.dropout1 = nn.Dropout(0.25)
        self.dropout2 = nn.Dropout(0.5)
        self.fc1 = nn.Linear(9216, 128)
        self.fc2 = nn.Linear(128, 10)

    def forward(self, x):
        x = self.conv1(x)
        x = F.relu(x)
        x = self.conv2(x)
        x = F.relu(x)
        x = F.max_pool2d(x, 2)
        x = self.dropout1(x)
        x = torch.flatten(x, 1)
        x = self.fc1(x)
        x = F.relu(x)
        x = self.dropout2(x)
        x = self.fc2(x)
        output = F.log_softmax(x, dim=1)
        return output

In [7]:
def train(log_interval, model, device, train_loader, optimizer, epoch):
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = F.nll_loss(output, target)
        loss.backward()
        optimizer.step()
        if batch_idx % log_interval == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader), loss.item()))

In [9]:
def test(model, device, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += F.nll_loss(output, target, reduction='sum').item()  # sum up batch loss
            pred = output.argmax(dim=1, keepdim=True)  # get the index of the max log-probability
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)

    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))

In [20]:
batch_size = 32
test_batch_size = 32
lr=1e-3
gamma=0.7
epochs=100
log_interval=50

In [21]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
train_kwargs = {'batch_size': batch_size}
test_kwargs = {'batch_size': test_batch_size}
if torch.cuda.is_available():
    cuda_kwargs = {'num_workers': 1,
                   'pin_memory': True,
                   'shuffle': True}
    train_kwargs.update(cuda_kwargs)
    test_kwargs.update(cuda_kwargs)

transform=transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.1307,), (0.3081,))
    ])
dataset1 = datasets.MNIST('../data', train=True, download=True,
                   transform=transform)
dataset2 = datasets.MNIST('../data', train=False,
                   transform=transform)
train_loader = torch.utils.data.DataLoader(dataset1,**train_kwargs)
test_loader = torch.utils.data.DataLoader(dataset2, **test_kwargs)

model = Net().to(device)
optimizer = optim.Adadelta(model.parameters(), lr=lr)

scheduler = StepLR(optimizer, step_size=1, gamma=gamma)
for epoch in range(1, epochs + 1):
    train(log_interval, model, device, train_loader, optimizer, epoch)
    test(model, device, test_loader)
    scheduler.step()

Train Epoch: 1 [0/60000 (0%)]	Loss: 2.270586
Train Epoch: 1 [1600/60000 (3%)]	Loss: 2.250265
Train Epoch: 1 [3200/60000 (5%)]	Loss: 2.227038
Train Epoch: 1 [4800/60000 (8%)]	Loss: 2.261923
Train Epoch: 1 [6400/60000 (11%)]	Loss: 2.231596
Train Epoch: 1 [8000/60000 (13%)]	Loss: 2.179885
Train Epoch: 1 [9600/60000 (16%)]	Loss: 2.145018
Train Epoch: 1 [11200/60000 (19%)]	Loss: 2.042520
Train Epoch: 1 [12800/60000 (21%)]	Loss: 2.126798
Train Epoch: 1 [14400/60000 (24%)]	Loss: 2.066688
Train Epoch: 1 [16000/60000 (27%)]	Loss: 2.107056
Train Epoch: 1 [17600/60000 (29%)]	Loss: 2.036979
Train Epoch: 1 [19200/60000 (32%)]	Loss: 1.907887
Train Epoch: 1 [20800/60000 (35%)]	Loss: 1.995362
Train Epoch: 1 [22400/60000 (37%)]	Loss: 1.870256
Train Epoch: 1 [24000/60000 (40%)]	Loss: 1.785676
Train Epoch: 1 [25600/60000 (43%)]	Loss: 1.852730
Train Epoch: 1 [27200/60000 (45%)]	Loss: 1.783761
Train Epoch: 1 [28800/60000 (48%)]	Loss: 1.654206
Train Epoch: 1 [30400/60000 (51%)]	Loss: 1.764339
Train Epoch: 1

KeyboardInterrupt: 